In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [13]:
# load training and test data
train = pd.read_csv('data/train_processed.csv')
test = pd.read_csv('data/test_processed.csv')

In [14]:
# preprocess skills feature

# determine unique skills
skills = set()
for m in train.skills:
    skills.update(g for g in m.split('|'))
skills = sorted(skills)

#make a column for each skill
for skill in skills:
    train[skill] = [int(skill in s.split('|')) for s in train.skills]

In [17]:
# do same calculation for test set as well
for skill in skills:
    test[skill] = [int(skill in s.split('|')) for s in test.skills]

In [19]:
# response variable
y = train.solved_status

## Prepare a very simple model based on just the accuracy of the problem

In [72]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [73]:
model1 = build_extreme_gradient_boosting(train, test)
model2 = build_random_forest_classifier(train, test)

In [74]:
scores = eval_models([model1, model2], train, y)

accuracy score: 0.855733
accuracy score: 0.854600
combined score: 0.852267
accuracy score: 0.856867
accuracy score: 0.854067
combined score: 0.852867
accuracy score: 0.858733
accuracy score: 0.851133
combined score: 0.852133
accuracy score: 0.859067
accuracy score: 0.855867
combined score: 0.855933
accuracy score: 0.846133
accuracy score: 0.855400
combined score: 0.853400


In [75]:
# train model
import time

start_time = time.time()
model1.fit(train, y)
model2.fit(train, y)
elapsed_time = time.time() - start_time

print 'It took %f seconds to train the model ' %(elapsed_time)

It took 35.870000 seconds to train the model 


In [78]:
predictions1  = model1.predict(test)
predictions2 = model2.predict(test)

ensemble_preds = np.array(majority_voting([predictions1, predictions2]))

In [77]:
# submissions
submission_df = pd.DataFrame({'Id': test.id.values, 'solved_status': ensemble_preds.astype(int)})
submission_df.to_csv('submissions/eleventh_xgb_rf.csv', index=False)

AttributeError: 'list' object has no attribute 'astype'

In [ ]:
user_problem_train.level.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()
lbl.fit(user_problem_train.level)

levels = lbl.transform(user_problem_train.level)

In [ ]:
pd.Series(levels).value_counts()